#1 Setup and imports

In [51]:
import sys
!{sys.executable} -m pip install pandas numpy matplotlib seaborn plotly scikit-learn >nul 2>&1
print("Install done")

Install done


In [ ]:
#Core libraries
import pandas as pd
import numpy as np 
#Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns 
import plotly.express as px 
#Machine learning libraries
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression, Ridge, LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import mean_absolute_error,mean_squared_error, classification_report, accuracy_score, confusion_matrix
#Data preprocessing libraries
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder

Matplotlib is building the font cache; this may take a moment.


#2 Data Collection

- *Clubs Data :*


Points, wins, draws, losses, goals for/against, xG/xGA

In [48]:
import pandas as pd
import warnings

warnings.filterwarnings("ignore")
seasons = [2025,2024,2023,2022,2021]
top5_standings = []

for season in seasons:
    url = f"https://fbref.com/en/comps/9/{season-1}-{season}/{season-1}-{season}-Premier-League-Stats"
    tables = pd.read_html(url)
    df = tables[0]
    df.columns = [' '.join(col).strip() if isinstance(col, tuple) else col for col in df.columns.values]

    #Select the first 5 teams (top 5 teams)
    top5 = df.head(5)
    top5.loc[:, 'Season'] = f"{season-1}/{season}"
    top5_standings.append(top5)

standings_df = pd.concat(top5_standings, ignore_index=True)
print(standings_df[['Season', 'Squad', 'Pts', 'W', 'D', 'L', 'GF', 'GA', 'GD','Rk']])

       Season            Squad  Pts   W   D   L  GF  GA  GD  Rk
0   2024/2025        Liverpool   84  25   9   4  86  41  45   1
1   2024/2025          Arsenal   74  20  14   4  69  34  35   2
2   2024/2025  Manchester City   71  21   8   9  72  44  28   3
3   2024/2025          Chelsea   69  20   9   9  64  43  21   4
4   2024/2025    Newcastle Utd   66  20   6  12  68  47  21   5
5   2023/2024  Manchester City   91  28   7   3  96  34  62   1
6   2023/2024          Arsenal   89  28   5   5  91  29  62   2
7   2023/2024        Liverpool   82  24  10   4  86  41  45   3
8   2023/2024      Aston Villa   68  20   8  10  76  61  15   4
9   2023/2024        Tottenham   66  20   6  12  74  61  13   5
10  2022/2023  Manchester City   89  28   5   5  94  33  61   1
11  2022/2023          Arsenal   84  26   6   6  88  43  45   2
12  2022/2023   Manchester Utd   75  23   6   9  58  43  15   3
13  2022/2023    Newcastle Utd   71  19  14   5  68  33  35   4
14  2022/2023        Liverpool   67  19 

Squad market value

In [60]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
# we will try to use requests with headers to avoid the automated blocking by the website
url = "https://www.transfermarkt.com/premier-league/startseite/wettbewerb/GB1/plus/?saison_id=2025"
headers = {"User-Agent": "Mozilla/5.0"}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "lxml")

table = soup.find("table", class_="items")
rows = table.find_all("tr", class_=["odd", "even"])

data = []
for row in rows:
    cells = row.find_all("td")
    if len(cells) >= 7:
        club = cells[1].get_text(strip=True)
        avg_value = cells[5].get_text(strip=True)
        total_value = cells[6].get_text(strip=True)
        data.append([club, avg_value, total_value])

sm_df = pd.DataFrame(data, columns=["Club", "Average Market Value", "Total Market Value"])
print(sm_df.head(10))


                     Club Average Market Value Total Market Value
0         Manchester City              €39.37m            €1.34bn
1              Chelsea FC              €28.92m            €1.30bn
2              Arsenal FC              €46.88m            €1.17bn
3            Liverpool FC              €37.19m            €1.08bn
4       Tottenham Hotspur              €25.00m           €849.90m
5       Manchester United              €25.54m           €817.20m
6        Newcastle United              €22.83m           €684.88m
7  Brighton & Hove Albion              €16.62m           €664.60m
8             Aston Villa              €19.30m           €559.80m
9          Crystal Palace              €15.38m           €461.30m


Manager changes

In [95]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://www.transfermarkt.com/premier-league/trainerwechsel/wettbewerb/GB1/plus/?saison_id=2024"
headers = {"User-Agent": "Mozilla/5.0"}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "lxml")

table = soup.find("table", class_="items")
rows = table.find_all("tr", class_=["odd", "even"])

data = []
for row in rows:
    # We only want rows with at least 7 cells
    cells = row.find_all("td")
    if len(cells) < 7:
        continue

    # We extract the club name from the <a> tag in the first cell
    # it's the title attribute of the <a> tag
    club_tag = cells[0].find("a")
    club = club_tag.get("title", club_tag.text.strip()) if club_tag else cells[0].get_text(strip=True)

    #The new manager located in the 11th cell
    #stripping the text to remove any extra spaces
    new_manager = cells[10].get_text(strip=True)

    data.append([club, new_manager])

managers_df = pd.DataFrame(data, columns=["Club", "New Manager"]).head(4)
print(managers_df)

                Club      New Manager
0       Brentford FC    Keith Andrews
1     Leicester City  Martí Cifuentes
2  Tottenham Hotspur     Thomas Frank
3     Southampton FC       Will Still


Transfers in/out (value, type)



Team strength metrics

- *Player Data :*

Goals, xG, shots, minutes played

Player position

Team attacking style

Penalty taker? (Yes/No)

In [45]:
# We will simply define a function to check either a player is a penalty taker or not and what is his order if sp
import requests

def is_penalty_taker(player_name):
    url = "https://fantasy.premierleague.com/api/bootstrap-static/"
    response = requests.get(url)
    
    if response.status_code != 200:
        raise Exception("Failed to fetch data from the API")
    data = response.json()
    for player in data['elements']:
        full_name = f"{player['first_name']} {player['second_name']}"
        if full_name.lower() == player_name.lower() or player['web_name'].lower() == player_name.lower() or player['first_name'].lower() == player_name.lower() or player['second_name'].lower() == player_name.lower():
            order = player['penalties_order']
            if order and order > 0: return ("Yes",order) 
            else: "No"
    return "Player not found"

is_penalty_taker("Minteh")  

'Player not found'

#3 Data Cleaning & Preprocessing

#4 Build the Prediction Models

#5 Evaluate Models

#6 Simulate Next Season

#7 Visualizations